In [109]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [110]:
df = pd.read_csv('../resources/owm09172017_09172018.csv')

/home/eric/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (10,11,20,23,25,39,40,44,45,59,63,64,67,68,84,86) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Overview of the dataset

In [83]:
df.head()

,STATION,STATION_NAME,ELEVATION,LATITUDE,LONGITUDE,DATE,REPORTTPYE,HOURLYSKYCONDITIONS,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,...,MonthlyMaxSeaLevelPressureTime,MonthlyMinSeaLevelPressureValue,MonthlyMinSeaLevelPressureDate,MonthlyMinSeaLevelPressureTime,MonthlyTotalHeatingDegreeDays,MonthlyTotalCoolingDegreeDays,MonthlyDeptFromNormalHeatingDD,MonthlyDeptFromNormalCoolingDD,MonthlyTotalSeasonToDateHeatingDD,MonthlyTotalSeasonToDateCoolingDD
0,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,-73.96925,2017-09-16 00:51,FM-15,CLR:00,7.00,NaN,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
1,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,-73.96925,2017-09-16 01:51,FM-15,CLR:00,6.00,BR:1 ||,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
2,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,-73.96925,2017-09-16 02:51,FM-15,CLR:00,4.00,BR:1 ||,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
3,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,-73.96925,2017-09-16 03:51,FM-15,CLR:00,3.00,BR:1 ||,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
4,WBAN:94728,NY CITY CENTRAL PARK NY US,42.7,40.77898,-73.96925,2017-09-16 03:59,FM-16,CLR:00,2.50,BR:1 ||,...,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
df.shape

(11959, 90)

In [85]:
df.columns

Index(['STATION', 'STATION_NAME', 'ELEVATION', 'LATITUDE', 'LONGITUDE', 'DATE',
       'REPORTTPYE', 'HOURLYSKYCONDITIONS', 'HOURLYVISIBILITY',
       'HOURLYPRSENTWEATHERTYPE', 'HOURLYDRYBULBTEMPF', 'HOURLYDRYBULBTEMPC',
       'HOURLYWETBULBTEMPF', 'HOURLYWETBULBTEMPC', 'HOURLYDewPointTempF',
       'HOURLYDewPointTempC', 'HOURLYRelativeHumidity', 'HOURLYWindSpeed',
       'HOURLYWindDirection', 'HOURLYWindGustSpeed', 'HOURLYStationPressure',
       'HOURLYPressureTendency', 'HOURLYPressureChange',
       'HOURLYSeaLevelPressure', 'HOURLYPrecip', 'HOURLYAltimeterSetting',
       'DAILYMaximumDryBulbTemp', 'DAILYMinimumDryBulbTemp',
       'DAILYAverageDryBulbTemp', 'DAILYDeptFromNormalAverageTemp',
       'DAILYAverageRelativeHumidity', 'DAILYAverageDewPointTemp',
       'DAILYAverageWetBulbTemp', 'DAILYHeatingDegreeDays',
       'DAILYCoolingDegreeDays', 'DAILYSunrise', 'DAILYSunset', 'DAILYWeather',
       'DAILYPrecip', 'DAILYSnowfall', 'DAILYSnowDepth',
       'DAILYAverageStatio

All data is, by design, from the Central Park station  
**Ideas**  
Cloud cover sunlight/present weather  
Could I do something neat with wind direction and speed? Some kind of cartesian coordinates with vectors for wind?  
Could choose daily data if hourly is too cluttered. Choose daily or monthly if displaying muliple years' worth of data

In [86]:
columns_keep = ['DATE', 'HOURLYSKYCONDITIONS', 'HOURLYVISIBILITY', 'HOURLYPRSENTWEATHERTYPE', 'HOURLYRelativeHumidity', 'HOURLYWindSpeed', 'HOURLYWindDirection',
               'HOURLYWindGustSpeed', 'HOURLYPrecip', 'DAILYSunrise', 'DAILYSunset', 'DAILYWeather', 'DAILYPrecip', 'DAILYSnowfall']

In [87]:
df_thin = df[columns_keep]

In [88]:
df_thin.head()

,DATE,HOURLYSKYCONDITIONS,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindDirection,HOURLYWindGustSpeed,HOURLYPrecip,DAILYSunrise,DAILYSunset,DAILYWeather,DAILYPrecip,DAILYSnowfall
0,2017-09-16 00:51,CLR:00,7.00,NaN,87.0,0.0,000,NaN,NaN,537,1804,NaN,NaN,NaN
1,2017-09-16 01:51,CLR:00,6.00,BR:1 ||,90.0,0.0,000,NaN,0.00,537,1804,NaN,NaN,NaN
2,2017-09-16 02:51,CLR:00,4.00,BR:1 ||,90.0,3.0,040,NaN,0.00,537,1804,NaN,NaN,NaN
3,2017-09-16 03:51,CLR:00,3.00,BR:1 ||,93.0,3.0,050,NaN,0.00,537,1804,NaN,NaN,NaN
4,2017-09-16 03:59,CLR:00,2.50,BR:1 ||,93.0,0.0,000,NaN,NaN,537,1804,NaN,NaN,NaN


Missing data

In [89]:
df_thin.isna().sum()

DATE                           0
HOURLYSKYCONDITIONS          393
HOURLYVISIBILITY             364
HOURLYPRSENTWEATHERTYPE     8792
HOURLYRelativeHumidity       362
HOURLYWindSpeed             1147
HOURLYWindDirection         1149
HOURLYWindGustSpeed        10162
HOURLYPrecip                1973
DAILYSunrise                   0
DAILYSunset                    0
DAILYWeather               11762
DAILYPrecip                11447
DAILYSnowfall              11595
dtype: int64

Available Data

In [90]:
df_thin.shape[0] - df_thin.isna().sum()

DATE                       11959
HOURLYSKYCONDITIONS        11566
HOURLYVISIBILITY           11595
HOURLYPRSENTWEATHERTYPE     3167
HOURLYRelativeHumidity     11597
HOURLYWindSpeed            10812
HOURLYWindDirection        10810
HOURLYWindGustSpeed         1797
HOURLYPrecip                9986
DAILYSunrise               11959
DAILYSunset                11959
DAILYWeather                 197
DAILYPrecip                  512
DAILYSnowfall                364
dtype: int64

Drop daily values for now

In [91]:
df_skinny = df_thin.drop(['DAILYWeather', 'DAILYPrecip', 'DAILYSnowfall'], axis=1)

In [92]:
df_skinny.shape[0] - df_skinny.isna().sum()

DATE                       11959
HOURLYSKYCONDITIONS        11566
HOURLYVISIBILITY           11595
HOURLYPRSENTWEATHERTYPE     3167
HOURLYRelativeHumidity     11597
HOURLYWindSpeed            10812
HOURLYWindDirection        10810
HOURLYWindGustSpeed         1797
HOURLYPrecip                9986
DAILYSunrise               11959
DAILYSunset                11959
dtype: int64

Create features for DATE parts

In [93]:
df_skinny['DATE'].head()

0    2017-09-16 00:51
1    2017-09-16 01:51
2    2017-09-16 02:51
3    2017-09-16 03:51
4    2017-09-16 03:59
Name: DATE, dtype: object

In [94]:
def get_year(row): # Get the year from DATE column
    return int(row[:4])
df_skinny['Year'] = df_skinny['DATE'].apply(get_year)

In [95]:
def get_month(row): # Get the month from DATE column
    return int(row[5:7])
df_skinny['Month'] = df_skinny['DATE'].apply(get_month)

In [96]:
def get_day(row): # Get the day from DATE column
    return int(row[8:10])
df_skinny['Day'] = df_skinny['DATE'].apply(get_day)

In [97]:
def get_hour(row): # Get the hour from DATE column
    return int(row[11:13])
df_skinny['Hour'] = df_skinny['DATE'].apply(get_hour)

Create matching features for sunrise and sunset

In [98]:
df_skinny['SunriseHour'] = df_skinny['DAILYSunrise'] // 100

In [99]:
df_skinny['SunriseMin'] = df_skinny['DAILYSunrise'] % 100

In [100]:
df_skinny['SunsetHour'] = df_skinny['DAILYSunset'] // 100

In [101]:
df_skinny['SunsetMin'] = df_skinny['DAILYSunset'] % 100

In [102]:
df_skinny['HOURLYSKYCONDITIONS'].head()

0    CLR:00
1    CLR:00
2    CLR:00
3    CLR:00
4    CLR:00
Name: HOURLYSKYCONDITIONS, dtype: object

In [208]:
def get_weather_type(row): #Get the weather type from HOURLYSKYCONDITIONS column
    if type(row) == str:
        search = re.findall(r'([A-Z]*)', row)
        return search[0]
    else:
        return 'NaN'

In [209]:
df_skinny['HourlySkyWeather'] = df_skinny['HOURLYSKYCONDITIONS'].apply(get_weather_type)

In [210]:
df_skinny['HourlySkyWeather'].head()

0    CLR
1    CLR
2    CLR
3    CLR
4    CLR
Name: HourlySkyWeather, dtype: object

In [211]:
set(df_skinny['HourlySkyWeather'])

{'BKN', 'CLR', 'FEW', 'NaN', 'OVC', 'SCT', 'VV'}

'10' which was supposed to denote partly cloudy, does that entry exist in the original feature? No

In [161]:
df_skinny['HOURLYSKYCONDITIONS'][df_skinny['HOURLYSKYCONDITIONS'].str.startswith('10', na=False)]

Series([], Name: HOURLYSKYCONDITIONS, dtype: object)

Could try to extract all weather types from HOURLYSKYCONDITIONS, but hold off until after looking at HOURLYPRSENTWEATHERTYPE

In [199]:
df_skinny['HOURLYVISIBILITY'].describe()

count     11595
unique       25
top       10.00
freq       7305
Name: HOURLYVISIBILITY, dtype: object

In [212]:
set(df_skinny['HOURLYVISIBILITY'])

{'0.00',
 '0.25',
 '0.50',
 '0.75',
 '0.75V',
 '1.00',
 '1.00V',
 '1.25',
 '1.25V',
 '1.50',
 '1.50V',
 '1.75',
 '1.75V',
 '10.00',
 '2.00',
 '2.00V',
 '2.50',
 '2.50V',
 '3.00',
 '4.00',
 '5.00',
 '6.00',
 '7.00',
 '8.00',
 '9.00',
 nan}

Remove the trailing V because the documentation does not provide a justification for it

In [219]:
def get_visibility(row): # Create tidy hourly visibility feature
    if type(row) == str:
        search = re.search(r'[0-9\.]*', row)
        return float(search[0])
    else:
        return 'NaN'

In [220]:
df_skinny['HourlyVisibility'] = df_skinny['HOURLYVISIBILITY'].apply(get_visibility)

In [221]:
set(df_skinny['HourlyVisibility'])

{0.0,
 0.25,
 0.5,
 0.75,
 1.0,
 1.25,
 1.5,
 1.75,
 10.0,
 2.0,
 2.5,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 'NaN'}

In [223]:
df_skinny['HOURLYPRSENTWEATHERTYPE'].summary()

count        3167
unique         37
top       BR:1 ||
freq         1035
Name: HOURLYPRSENTWEATHERTYPE, dtype: object

HOURLYPRSENTWEATHERTYPE is formatted as AU|AW|MW where AU and AW are automatic weather type measurements and MW is a manual measurement

In [224]:
set(df_skinny['HOURLYPRSENTWEATHERTYPE'])

{'+RA:02 BR:1 |RA:63 |',
 '+RA:02 BR:1 |RA:63 |RA:63',
 '+RA:02 FG:2 |FG:30 RA:63 |',
 '+RA:02 FG:2 |FG:30 RA:63 |RA:63',
 '+RA:02 |RA:63 |',
 '+RA:02 |RA:63 |RA:63',
 '+SN:03 FG:2 |FG:30 SN:73 |',
 '+SN:03 FZ:8 FG:2 |FG:35 SN:73 |',
 '-FZ:8 RA:02 BR:1 |FZRA:64 |',
 '-RA:02 BR:1 |RA:61 |',
 '-RA:02 BR:1 |RA:61 |RA:61',
 '-RA:02 FG:2 |FG:30 RA:61 |RA:61',
 '-RA:02 |RA:61 |',
 '-RA:02 |RA:61 |RA:61',
 '-RA:02 ||',
 '-SN:03 BR:1 |SN:71 |',
 '-SN:03 FZ:8 FG:2 |FG:35 SN:71 |',
 '-SN:03 |SN:71 |',
 'BR:1 ||',
 'FG:2 |FG:30 |',
 'FZ:8 RA:02 BR:1 |FZRA:65 |',
 'HZ:7 |FU:05 |HZ:05',
 'HZ:7 ||HZ:05',
 'RA:02 BR:1 |RA:62 |',
 'RA:02 BR:1 |RA:62 |RA:65',
 'RA:02 FG:2 |FG:30 RA:62 |RA:65',
 'RA:02 |RA:62 s |RA:65 s',
 'RA:02 |RA:62 |',
 'RA:02 |RA:62 |RA:65',
 'SN:03 FG:2 |FG:30 SN:72 |',
 'SN:03 FZ:8 FG:2 |FG:35 SN:72 |',
 'SN:03 |SN:72 s |',
 'UP:09 BR:1 ||',
 'UP:09 ||',
 nan,
 '|RA:61 |',
 '|RA:61 |RA:61',
 '|SN:71 |'}

In [108]:
df_skinny.columns

Index(['DATE', 'HOURLYSKYCONDITIONS', 'HOURLYVISIBILITY',
       'HOURLYPRSENTWEATHERTYPE', 'HOURLYRelativeHumidity', 'HOURLYWindSpeed',
       'HOURLYWindDirection', 'HOURLYWindGustSpeed', 'HOURLYPrecip',
       'DAILYSunrise', 'DAILYSunset', 'Year', 'Month', 'Day', 'Hour',
       'SunriseHour', 'SunriseMin', 'SunsetHour', 'SunsetMin',
       'HourlySkyWeather'],
      dtype='object')